# ECON 8185 - HW 2

Author: João Lazzaro - santo279@umn.edu

## Question 1

We'll consider the following Prototype model from Ellen's Homework 2, which is the same as CKM:

$$\max_{c_t,x_t,l_t} E \sum_{t=0}^\infty \beta^t \frac{\left(c_tl_t^\psi\right)^{1-\sigma}}{1-\sigma} N_t$$
S.T. 
$$ c_t +(1+\tau_{xt}x_t =r_t k_t +(1-\tau_{ht})w_t h_t + T_t$$
$$ N_{t+1} k_{t+1} = [(1-\delta)k_t+x_t]N_t$$
$$h_t+l_t =1$$
$$S_t = P S_{t-1}+Q\epsilon_t,\;\;S_t = [\ln z_t,\tau_{ht},\tau_{xt}, \ln g_t]$$
$$ c_t,x_t\geq0$$

Where $N_t = (1+\gamma_n)^t$ and firm tecnhology is $Y_t = K_t^\theta (Z_t L_t)^{1-\theta}$. $\gamma_z$ is the rate of labor-augmenting technical progress. The stochastic shocks hitting this economy affect technology, tax rates, and government spending and the stochastic processes are modeled as a VAR(1) process. The resource constraint is $Y_t=N_t(c_t +x_t +g_t)$. We must work with detrended variables, we should use "hat" variables, but typing this is time consuming so from now on all variables should be understood as detrended unless I say something about them.

The detrended FOC's of this model are:

$$c_t+(1+\gamma_z)(1+\gamma_n)k_{t+1} - (1-\delta)k_t+g_t = y_t =k_t^\theta(z_th_t)^{1-\theta}$$
$$\psi \frac{c_t}{1-h_t} = (1-\tau_{ht})(1-\theta)\left(\frac{k_t}{h_t}\right)^\theta z_t^{1-\theta}$$
$$c_t^{-\sigma}(1-h_t)^{\psi(1-\sigma)} (1+\tau_{xt})=\beta(1+\gamma_z)^{-\sigma}E_t c_{t+1}^{-\sigma} (1- h_{t+1})^{\psi(1-\sigma)}\left(\theta k_{t+1}^{\theta}(z_{t+1} h_{t+1})^{1-\theta} + (1-\delta)(1+\theta_{xt+1})\right)$$

Defining some parameters:

In [3]:
using Plots, NLsolve, ForwardDiff, DataFrames


#Parameters:
δ = 0.05    #depreciation rate
θ = 1/3  #capital share of output
β = 0.975  #Discouting
σ = 1  #Elasticity of Intertemporal Substitution
ψ = 2    #Labor parameter
γn= 0.003    #Population growth rate
γz= 0.004   #Productivitu growth rate
gss = 0.03 #average g
τxss = 0.01 #average τx
τhss = 0.02 #average τh
zss = 1 #average z



1

Substituting for $c$, we get an euqation for $k$, and one for $h$. Below, I find the Steady State values:


In [4]:
#Function with the FOCs
function SS!(eq, vector::Vector)
    k,h = (vector)
    k1 = k
    h1 = h
    g, τx,τh, z = gss,τxss,τhss, zss
    z1 = z
    τx1 = τx
    c = k^θ * (z *h)^(1-θ) - ((1+γz)*(1+γn)*k1-(1-δ)*k+g)
    c1 = c
    eq[1] = ψ *c/(1-h) - (1-τh)*(1-θ) *(k/h)^θ *z^(1-θ)
    eq[2] = c^(-σ) *(1-h)^(ψ*(1-σ))*(1+τx)  -  β*(1+γz)^(-σ) * c1^(-σ) * (1-h1)^(ψ*(1-σ)) *(θ*k1^(-θ)*(z1*h1)^(1-θ)+(1-δ)*(1+τx1))
    return eq
end

SteadyState = nlsolve(SS!, [0.05,0.97],ftol = :1.0e-20, method = :trust_region , autoscale = true)
kss,hss = SteadyState.zero

2-element Array{Float64,1}:
 13.889514153222507 
  0.4426404952076869

Log-linearizing the equations we get the following system of equations:


$$ 0 = E_t[a_1\tilde{k}_t +a_2\tilde{k}_{t+1}+a_3\tilde{h}_t+a_4 \tilde{z}_t + a_5 \tilde{\tau}_{ht} +a_6 \tilde{g}_t] $$
$$ 0 = E_t[b_1\tilde{k}_t +b_2\tilde{k}_{t+1}+b_3\tilde{k}_{t+2}+b_4 \tilde{h}_t + b_5 \tilde{h}_{t+1} b_6 \tilde{z}_t +b_7 \tilde{\tau}_{xt} +b_8 \tilde{g}_t + b_9 \tilde{z}_{t+1} +b_{10} \tau_{xt+1}+b_{11}\tilde{g}_{t+1}]  $$

Where tilde variables are log deviations from Steady State.